In [1]:
#normalize, categorize, dummify features in dataframe
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
import pandas as pd
import numpy as np
import operator
import pickle

#read in pickled final movie dataframe
with open('finalMovie_df.pkl', 'r') as picklefile:
    finalMovie_df = pickle.load(picklefile)

#drop columns, lack of sufficient data, unimportant, some maybe I could come back around to
finalMovie_df.drop('tomatoURL', axis=1, inplace=True)
finalMovie_df.drop('BoxOffice', axis=1, inplace=True)
finalMovie_df.drop('Writer', axis=1, inplace=True)
finalMovie_df.drop('Website', axis=1, inplace=True)
finalMovie_df.drop('tomatoConsensus', axis=1, inplace=True)
finalMovie_df.drop('tomatoImage', axis=1, inplace=True)
finalMovie_df.drop('Poster', axis=1, inplace=True)
finalMovie_df.drop('Metascore', axis=1, inplace=True)
finalMovie_df.drop('Response', axis=1, inplace=True)
finalMovie_df.drop('Released', axis=1, inplace=True)
finalMovie_df.drop('tomatoUserReviews', axis=1, inplace=True)
finalMovie_df.drop('DVD', axis=1, inplace=True)
finalMovie_df.drop('imdbID', axis=1, inplace=True)
finalMovie_df.drop('Type', axis=1, inplace=True)
finalMovie_df.drop('Production', axis=1, inplace=True)
finalMovie_df.drop('tomatoFresh', axis=1, inplace=True)
finalMovie_df.drop('tomatoReviews', axis=1, inplace=True)
finalMovie_df.drop('tomatoRotten', axis=1, inplace=True)
finalMovie_df.drop('tomatoUserMeter', axis=1, inplace=True)


In [2]:
#standardize imdbRating

#define function to convert string to float
def string2float(x):
    try:
        x = float(x)
        return x
    except:
        return x
    
#only a few hundred missing data points, sample from the rest of the movies
def sample(x):
    while x == 'N/A':
        y = finalMovie_df['imdbRating'].sample()
        try:
            y = float(y.iloc[0])
            x = y
            return x
        except:
            continue
    else:
        return x
    
#apply functions to column and scale data
finalMovie_df['imdbRating'] = finalMovie_df['imdbRating'].apply(string2float).apply(sample)

#create dataframe with features that need null values populated
testdata_df = finalMovie_df[['tomatoMeter', 'imdbRating', 'tomatoUserRating']]

#write pickle file to use in other notebook
with open('testdata_df.pkl', 'w') as picklefile:
    pickle.dump(testdata_df, picklefile)

In [3]:
#standardize year(not classify/dummify)
finalMovie_df['Year'] = finalMovie_df['Year'].str[0:4].astype(int)
finalMovie_df['Year'] = preprocessing.scale(finalMovie_df['Year'])

//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [4]:
#Genre

#split by commas and if 'N/A' return 'No Genre', ordered by importance so take first genre listed
def genresplit(x):
    y = x.split(",")
    if y[0] == 'N/A':
        return 'No Genre'
    else:
        return y[0]

#take second genre, if no second genre repeat the first
def genresplit2(x):
    y = x.split(",")
    if y[1] == None:
        return y[0]
    else:
        return y[1]

#apply the functions to 'Genre' column and create two new columns, then dummify and 
#add back to original dataframe
finalMovie_df['Genre-1'] = finalMovie_df['Genre'].apply(genresplit)
finalMovie_df['Genre-2'] = finalMovie_df['Genre'].apply(genresplit)
Genre1dummy = pd.get_dummies(finalMovie_df['Genre-1'])
Genre2dummy = pd.get_dummies(finalMovie_df['Genre-2'])
finalMovie_df = pd.concat([finalMovie_df, Genre1dummy, Genre2dummy], axis=1)
finalMovie_df.drop('Genre', axis=1, inplace=True)

In [5]:
#Language
#people like domestic or international movies, or maybe have particular affinity to a well known
#countrys' movies but more exotic cinema can be grouped together
#exist fanatics of Spanish language film but not Xhosa language'
languages = ['Arabic','Bulgarian','Chinese','Croatian',\
             'Dutch', 'English', 'Finnish', 'French','German',\
             'Greek','Hebrew','Hindi', 'Hungarian', 'Icelandic',\
             'Italian', 'Japanese', 'Korean', 'Norwegian',\
             'Persian','Polish', 'Portuguese', 'Punjabi',\
             'Romanian', 'Russian', 'Spanish','Swedish', \
             'Turkish', 'Ukrainian']

#split languages by comma, if not in major language list return 'secondary language'
def languagesplit(x):
    y = x.split(",")
    if y[0] == 'N/A':
        return 'No Language'
    else:
        if y[0] in languages:
            return y[0]
        else:
            return 'Secondary language'

#repeat process again for second language, if none listed repeat the first
def languagesplit2(x):
    y = x.split(",")
    if len(y) < 2:
        if y[0] == 'N/A':
            return 'No Language'
        else:
            if y[0] in languages:
                return y[0]
            else:
                return 'Secondary language'
    else:
        if y[1] in languages:
            return y[1]
        else:
            return 'Secondary language'

finalMovie_df['Language-1'] = finalMovie_df['Language'].apply(languagesplit)
finalMovie_df['Language-2'] = finalMovie_df['Language'].apply(languagesplit2)
Language1dummy = pd.get_dummies(finalMovie_df['Language-1'])
Language2dummy = pd.get_dummies(finalMovie_df['Language-2'])
finalMovie_df = pd.concat([finalMovie_df, Language1dummy, Language2dummy], axis=1)
finalMovie_df.drop('Language', axis=1, inplace=True)

In [6]:
#Country-same idea as language
maincountries = ['Argentina', 'Australia', 'Austria', 'Belgium', \
                 'Brazil', 'Bulgaria', 'Canada', 'China', 'Chile',\
                 'Colombia', 'Costa Rica', 'Czech Republic', 'Denmark',\
                 'Finland', 'France', 'Germany', 'Greece','Hong Kong',\
                 'Hungary', 'Iceland', 'India','Iran', 'Ireland', 'Italy',\
                 'Japan','Malaysia', 'Mexico', 'Netherlands', 'New Zealand',\
                 'Pakistan', 'Poland', 'Portugal', 'Romania','Russia',\
                 'Singapore', 'South Africa', 'Spain','Sweden',\
                 'Switzerland', 'Thailand', 'UK','USA', 'Soviet Union', 'West Germany']

def countrysplit(x):
    y = x.split(",")
    if y[0] == 'N/A':
        return 'No Country'
    else:
        if y[0] in maincountries:
            return y[0]
        else: 
            return 'Small market country'
    
def countrysplit2(x):
    y = x.split(",")
    if len(y) < 2:
        if y[0] == 'N/A':
            return 'No Country'
        else:
            if y[0] in maincountries:
                return y[0]
            else:
                return 'Small market country'
    else:
        if y[1] in maincountries:
            return y[1]
        else:
            return 'Small market country'    
    

finalMovie_df['Country-1'] = finalMovie_df['Country'].apply(countrysplit)
finalMovie_df['Country-2'] = finalMovie_df['Country'].apply(countrysplit2)
Country1dummy = pd.get_dummies(finalMovie_df['Country-1'])
Country2dummy = pd.get_dummies(finalMovie_df['Country-2'])
finalMovie_df = pd.concat([finalMovie_df, Country1dummy, Country2dummy], axis=1)

finalMovie_df.drop('Country', axis=1, inplace=True)

In [7]:
#Actors- using a moviestar factor, how many movies has actor been in throughout entire dataframe
#people watch movies not for specific actor (this is my supposition) but if it is big budget vs.
#independent movie

#as before, split actors on comma
def actorsplit(x):
    y = x.split(",")
    if y[0] == 'N/A':
        return 'No Actor'
    else:
        return y[0]

def actorsplit2(x):
    y = x.split(",")
    if len(y) < 2:
        return y[0]
    else:
        return y[1]

#use groupby function to sum up amount of movies in which actor has appeared
#use lambda function to then assign this value to new columns
finalMovie_df['Actor-Lead'] = finalMovie_df['Actors'].apply(actorsplit)
finalMovie_df['Actor-LeadSQ'] = finalMovie_df.groupby('Actor-Lead')['Actor-Lead'].transform('count')
finalMovie_df['Actor-Lead'] = finalMovie_df.apply(lambda row: row['Actor-LeadSQ'], axis=1)
finalMovie_df['Actor-Support'] = finalMovie_df['Actors'].apply(actorsplit2)
finalMovie_df['Actor-SupportSQ'] = finalMovie_df.groupby('Actor-Support')['Actor-Support'].transform('count')
finalMovie_df['Actor-Support'] = finalMovie_df.apply(lambda row: row['Actor-SupportSQ'], axis=1)
finalMovie_df.drop('Actors', axis=1, inplace=True)

#still need to normalize these values
finalMovie_df['Actor-Lead'] = preprocessing.scale(finalMovie_df['Actor-Lead'])
finalMovie_df['Actor-Support'] = preprocessing.scale(finalMovie_df['Actor-Support'])

In [8]:
#Directors

def directorsplit(x):
    y = x.split(",")
    if y[0] == 'N/A':
        return 'No Director'
    else:
        return y[0]


finalMovie_df['Director'] = finalMovie_df['Director'].apply(directorsplit)
finalMovie_df.groupby('Director').count()

#to avoid having unimportant features, group one-time directors into one feature
finalMovie_df['DirectorSQ'] = finalMovie_df.groupby('Director')['Director'].transform('count')
finalMovie_df['Director'] = finalMovie_df.apply(lambda row: 'one-timer' if row['DirectorSQ'] == 1 else row['Director'], axis=1)

#dummify the column
Directordummy = pd.get_dummies(finalMovie_df['Director'])
finalMovie_df = pd.concat([finalMovie_df, Directordummy], axis=1)

finalMovie_df.drop('Director', axis=1, inplace=True)

In [9]:
#Awards                             
#number of wins
#number of columns
#pull out number of award nominations for one column and then number of wins in another
#also have the option of using amount of Oscar wins, Emmy wins, etc.

def wins_split(x):
    y = x.split(" ")
    if len(y) > 1:
        try: 
            y[1] == 'win'
            return int(y[0])
        except:
            try:
                y[3] == 'Another'
                z = int(y[4]) + int(y[1])
                return z
            except:
                try:
                    y[4] == 'Another'
                    z = int(y[5]) + int(y[1])
                    return z
                except:
                    return 0
    else: 
        return 0

def nom_split(x):
    y = x.split(" ")
    if len(y) > 1:
        try: 
            y[1] == 'nomination'
            return int(y[0])
        except:
            try:
                y[2] == '&'
                z = int(y[3])
                return z
            except:
                try:
                    y[8] == 'nominations'
                    z = int(y[7])
                    return z
                except:
                    try:
                        y[9] == 'nominations'
                        z = int(y[8])
                        return z
                    except:
                        return 0
    else: 
        return 0

finalMovie_df['Award_wins'] = finalMovie_df['Awards'].apply(wins_split)
finalMovie_df['Award_wins'] = preprocessing.scale(finalMovie_df['Award_wins'])
finalMovie_df['Award_noms'] = finalMovie_df['Awards'].apply(nom_split)
finalMovie_df['Award_noms'] = preprocessing.scale(finalMovie_df['Award_noms'])
finalMovie_df.drop('Awards', axis=1, inplace=True)

In [10]:
#Rated
#Does N/A have some information in it
#finalMovie_df.groupby('Rated').count()

In [11]:
#imdbVotes

def sample(x):
    while x == 'N/A':
        y = finalMovie_df['imdbVotes'].sample()
        try:
            y = y.iloc[0]
            x = float(y.replace(',',''))
            return x
        except:
            continue
    else:
        x = float(x.replace(',',''))
        return x
finalMovie_df['imdbVotes'] = finalMovie_df['imdbVotes'].apply(sample)
finalMovie_df['imdbVotes'] = preprocessing.scale(finalMovie_df['imdbVotes'])


In [12]:
#Runtime
#Convert from hours to minutes where needed, normailize

def hour2min(x):
    y = x.split(' ')
    if len(y) > 1:
        if y[1] == 'min':
            try:
                minutes = float(y[0])
            except:
                minutes = y[0]
        elif y[1] == 'h':
            minutes = float(y[0])*60
            minutes = minutes + float(y[2])
        else:
            minutes = 'N/A'
        return minutes
    else:
        return 'N/A'
    
def sample(x):
    while x == 'N/A':
        y = finalMovie_df['Runtime'].sample()
        try: 
            y = y.iloc[0]
            if y == 'N/A':
                continue
            else:
                return y
        except:
            continue
    else:
        return x
    
def tofloat(x):
    if type(x) != float:
        try:
            x = float(x.replace(',',''))
            return x
        except:
            print 'nope'
    else:
        return x

finalMovie_df['Runtime'] = finalMovie_df['Runtime'].apply(sample).apply(hour2min).apply(tofloat)
finalMovie_df['Runtime'] = preprocessing.scale(finalMovie_df['Runtime'])

In [13]:
#import modules to use for NLP that will be applied to the plot summary

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

#perform tf-idf on all of the plot summaries and fit/transform the sparse matrix

tfidf = TfidfVectorizer(stop_words = 'english', strip_accents = 'unicode')
X = tfidf.fit_transform(list(finalMovie_df['Plot']))


In [21]:
#create instance of SVD class, similar to Principal Component Analysis,
#Singular Value Decomposition is a factorization of a real or complex matrix
#and in this case used as an alternative form of dimensionality reduction 
#because matrix is sparse

svd = TruncatedSVD(n_components= 5000)

In [ ]:
#fit and transform matrix using SVD, rotating axes and transforming to original 'form'
#of my dataframe (with dimensions equal to number of components specified above)

x = svd.fit_transform(X)

In [ ]:
#print shape of matrix to confirm correct form

x.shape

In [17]:
#explained variance ratio, amount of variance maintained after performing SVD
#and reducing dimensions

svd.explained_variance_ratio_ .sum()

0.25388595715493012

In [18]:
SVD = pd.DataFrame(x)

In [19]:
with open('populatednullsfinal_df.pkl', 'r') as picklefile:
    populatednullsfinal_df = pickle.load(picklefile)

finalMovie_df.drop('tomatoMeter', axis=1, inplace=True)
finalMovie_df.drop('imdbRating', axis=1, inplace=True)
finalMovie_df.drop('tomatoUserRating', axis=1, inplace=True)
finalMovie_df.drop('tomatoRating', axis=1, inplace=True)

finalMovienorm_df = pd.concat([finalMovie_df, populatednullsfinal_df, SVD], axis=1)
finalMovienorm_df

,movieId,Plot,Rated,Title,Runtime,imdbVotes,tomatoRating,Year,netflix,hulu,...,490,491,492,493,494,495,496,497,498,499
0,1.0,A cowboy doll is profoundly threatened and jea...,G,Toy Story,-0.536046,9.048695,9.0,0.120776,0,0,...,0.000426,-0.005778,-0.030734,0.011034,0.022177,0.001988,0.017453,0.010669,0.014786,-0.034015
1,2.0,When two kids find and play a magical board ga...,PG,Jumanji,0.224695,2.812747,5.6,0.120776,0,0,...,0.001594,0.003728,-0.001281,-0.010732,0.012791,-0.017731,0.012486,-0.003717,0.014483,0.011170
2,3.0,John and Max resolve to save their beloved bai...,PG-13,Grumpier Old Men,0.125468,-0.000271,4.2,0.120776,0,0,...,-0.011194,-0.000856,0.011177,-0.026737,0.010572,-0.004720,-0.009283,-0.022856,-0.000424,0.000116
3,4.0,"Based on Terry McMillan's novel, this film fol...",R,Waiting to Exhale,0.886209,-0.173651,5.6,0.120776,0,0,...,-0.000675,0.030805,-0.012675,-0.009919,-0.008326,0.003666,0.018595,-0.009816,-0.017357,0.002553
4,5.0,George Banks must deal not only with the pregn...,PG,Father of the Bride Part II,0.290846,0.109904,5.3,0.120776,0,0,...,0.002365,0.034729,-0.016516,0.005364,-0.006923,-0.020569,-0.011453,-0.003333,-0.015061,-0.000238
5,6.0,A group of professional bank robbers start to ...,R,Heat,2.407691,6.131345,7.8,0.120776,0,0,...,-0.013214,-0.016301,0.020690,-0.008894,-0.007306,0.005310,-0.003373,-0.017336,0.001634,-0.018725
6,7.0,An ugly duckling having undergone a remarkable...,PG,Sabrina,0.985436,0.153487,6.0,0.120776,0,0,...,0.000818,0.003239,-0.007096,-0.010381,0.001566,0.003268,-0.014795,-0.002491,0.007959,-0.005194
7,8.0,Tom and Huck witness Injun Joe's killing of Do...,PG,Tom and Huck,-0.006835,-0.173000,4.2,0.120776,0,0,...,-0.010513,0.013640,-0.010124,-0.012909,0.011515,-0.012271,-0.003976,-0.015556,0.003443,0.003198
8,9.0,A former fireman takes on a group of terrorist...,R,Sudden Death,0.456225,0.096065,5.1,0.120776,0,0,...,-0.004987,0.001529,0.017739,0.001091,-0.012490,-0.017772,-0.005327,0.001943,-0.005749,0.024132
9,10.0,James Bond teams up with the lone survivor of ...,PG-13,GoldenEye,1.084663,2.633901,7.0,0.120776,0,1,...,-0.013385,0.009443,-0.010778,0.012628,0.055439,-0.036854,0.054722,0.009532,0.040846,-0.004571


In [20]:
with open('finalMovienorm_df.pkl', 'w') as picklefile:
    pickle.dump(finalMovienorm_df, picklefile)